In [4]:
!pip show pyspark

Name: pyspark
Version: 4.0.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: C:\Users\ADIPAKSH\AppData\Roaming\Python\Python311\site-packages
Requires: py4j
Required-by: 


In [23]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col
from pyspark import SparkContext

#10th code:-Department wise weekly sales(Aviral)

# Step 1: Initialize SparkSession
spark = SparkSession.builder \
    .appName("WeeklySalesByDepartment") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

# Step 2: Initialize SparkContext
sc = SparkContext.getOrCreate()
spark.sparkContext.setLogLevel("INFO")

# Step 3: Load CSV as DataFrame
df = spark.read.option("header", True).option("inferSchema", True).csv(
    "C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/train.csv"
)

# Step 4: Ensure correct data types
df = df.withColumn("Weekly_Sales", col("Weekly_Sales").cast("double"))

# Step 5: Group by Department and calculate total weekly sales
result_df = df.groupBy("Dept") \
    .agg(sum("Weekly_Sales").alias("Total_Weekly_Sales")) \
    .orderBy("Dept")

# Step 6: Show result
result_df.show()

# Step 7: Save results in multiple formats
result_df.write.mode("overwrite").csv("C:/Users/ADIPAKSH/Desktop/output10-2/weekly_sales_by_department.csv")
result_df.write.mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/output10-2/weekly_sales_by_department_parquet")
result_df.write.mode("overwrite").orc("C:/Users/ADIPAKSH/Desktop/output10-2/weekly_sales_by_department_orc")

# Step 8: Partition by Department
result_df.write.partitionBy("Dept").mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/output10-2/partitioned_by_department.csv")


+----+--------------------+
|Dept|  Total_Weekly_Sales|
+----+--------------------+
|   1|1.2363877654000005E8|
|   2|2.8061117443000007E8|
|   3| 7.589244995000002E7|
|   4|1.6714674557999995E8|
|   5|1.3560735857000008E8|
|   6|2.8420667139999993E7|
|   7|1.5547756275000027E8|
|   8| 1.942807807300001E8|
|   9|1.2839325665000004E8|
|  10|1.1789738758000001E8|
|  11| 9.332927637999998E7|
|  12| 2.676011951000001E7|
|  13|1.9732156995000008E8|
|  14| 9.569466644999984E7|
|  16| 9.167068226999989E7|
|  17| 6.531981662999996E7|
|  18| 3.689748906000001E7|
|  19|   6816183.110000005|
|  20| 3.334964510999999E7|
|  21| 3.206464508000002E7|
+----+--------------------+
only showing top 20 rows



In [24]:

#9th code:- Weekly sales by CPI(Aviral)
# Step 1: Initialize SparkSession
spark = SparkSession.builder \
    .appName("WeeklySalesByCPI") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")

# Step 2: Load train.csv as DataFrame
train_df = spark.read.option("header", True).option("inferSchema", True).csv(
    "C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/train.csv"
)

# Step 3: Load features.csv to get CPI
features_df = spark.read.option("header", True).option("inferSchema", True).csv(
    "C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/features.csv"
)

# Step 4: Join train data with CPI
joined_df = train_df.join(features_df.select("Store", "Date", "CPI"), on=["Store", "Date"])

# Step 5: Ensure Weekly_Sales is cast to double
joined_df = joined_df.withColumn("Weekly_Sales", col("Weekly_Sales").cast("double"))

# Step 6: Group by CPI and calculate total weekly sales
result_df = joined_df.groupBy("CPI") \
    .agg(sum("Weekly_Sales").alias("Total_Weekly_Sales")) \
    .orderBy("CPI")

# Step 7: Show result
result_df.show()

# Step 8: Write output to local file system
result_df.write.mode("overwrite").csv("C:/Users/ADIPAKSH/Desktop/output10-1/weekly_sales_by_cpi.csv")

# Step 9: Write as Parquet and ORC
result_df.write.mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/output10-1/weekly_sales_by_cpi_parquet")
result_df.write.mode("overwrite").orc("C:/Users/ADIPAKSH/Desktop/output10-1/weekly_sales_by_cpi_orc")

# Step 10: Partition by CPI
result_df.write.partitionBy("CPI").mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/output10-1/partitioned_by_cpi")


+-----------+--------------------+
|        CPI|  Total_Weekly_Sales|
+-----------+--------------------+
|    126.064|1.1186672609999998E7|
|126.0766452|1.1236542210000005E7|
|126.0854516|       1.161712406E7|
|126.0892903|       1.064440361E7|
|126.1019355|1.1446981889999995E7|
|126.1069032|1.0543480790000001E7|
|126.1119032|       1.147065679E7|
|    126.114|1.1245237399999999E7|
|126.1145806|1.1122414159999998E7|
|   126.1266|1.0918034249999996E7|
|126.1283548|       1.086616566E7|
|126.1360645|1.1793210660000004E7|
|   126.1392|1.1412042209999999E7|
|126.1454667|1.0575165890000004E7|
|126.1498065|1.1082464319999998E7|
|   126.1518|1.0981588900000002E7|
|126.1602258|1.0968939939999998E7|
|126.1843871|       1.092779466E7|
|126.1900333|1.0191666909999998E7|
|126.2085484|       1.106177064E7|
+-----------+--------------------+
only showing top 20 rows



In [25]:
#Code 1:- Weekly sales by Store and Holiday(Anagha)
from pyspark.sql.functions import sum, col, avg

# Step 1: Initialize SparkSession
spark = SparkSession.builder \
    .appName("WeeklySalesByStoreHoliday") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")

# Step 2: Load CSV as RDD
rdd = spark.sparkContext.textFile("C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/train.csv")

# Step 3: Convert RDD to DataFrame
header = rdd.first()
data_rdd = rdd.filter(lambda row: row != header).map(lambda row: row.split(","))
df = data_rdd.map(lambda arr: (int(arr[0]), int(arr[1]), arr[2], float(arr[3]), arr[4] == 'TRUE')) \
    .toDF(["Store", "Dept", "Date", "Weekly_Sales", "IsHoliday"])

# Step 4: Group by Store and IsHoliday
result_df = df.groupBy("Store", "IsHoliday") \
    .agg(sum("Weekly_Sales").alias("Total_Weekly_Sales")) \
    .orderBy("Store", "IsHoliday")

# Step 5: Show result
result_df.show()

# Step 6: Write output to local file system
result_df.write.mode("overwrite").csv("C:/Users/ADIPAKSH/Desktop/output1-1/weekly_sales_by_store_holiday.csv")

# Step 7: Write as Parquet and ORC
result_df.write.mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/output1-1/weekly_sales_by_store_holiday_parquet")
result_df.write.mode("overwrite").orc("C:/Users/ADIPAKSH/Desktop/output1-1/weekly_sales_by_store_holiday_orc")

# Step 8: Partition DF by IsHoliday
result_df.write.partitionBy("IsHoliday").mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/output1-1/partitioned_by_holiday")


# Optional: Stop Spark session
# spark.stop()


+-----+---------+--------------------+
|Store|IsHoliday|  Total_Weekly_Sales|
+-----+---------+--------------------+
|    1|    false|2.0574533228999949E8|
|    1|     true|1.6657476559999997E7|
|    2|    false|2.5458977198000062E8|
|    2|     true|2.0792669000000022E7|
|    3|    false| 5.320862456999985E7|
|    3|     true|           4378110.5|
|    4|    false| 2.771129271399988E8|
|    4|     true|2.2431026240000024E7|
|    5|    false|  4.18806728300003E7|
|    5|     true|   3595016.070000001|
|    6|    false|2.0694705137000006E8|
|    6|     true|1.6809079270000003E7|
|    7|    false|       7.487427249E7|
|    7|     true|   6724002.650000009|
|    8|    false|1.2019787252999978E8|
|    8|     true|   9753308.600000001|
|    9|    false| 7.189971077999982E7|
|    9|     true|   5889508.209999993|
|   10|    false| 2.504801543999997E8|
|   10|     true|2.1137559489999976E7|
+-----+---------+--------------------+
only showing top 20 rows



In [26]:
#Code 2:- weekly sales by temperature and year(Anagha)
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, year, to_date

# Step 1: Initialize SparkSession
spark = SparkSession.builder \
    .appName("WeeklySalesByTempYear") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")

# Step 2: Load train.csv as RDD
rdd = spark.sparkContext.textFile("C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/train.csv")

# Step 3: Convert RDD to DataFrame
header = rdd.first()
data_rdd = rdd.filter(lambda row: row != header).map(lambda row: row.split(","))
df = data_rdd.map(lambda arr: (int(arr[0]), int(arr[1]), arr[2], float(arr[3]), arr[4] == 'TRUE')) \
    .toDF(["Store", "Dept", "Date", "Weekly_Sales", "IsHoliday"])

# Step 4: Load features.csv and join
features_df = spark.read.option("header", True).option("inferSchema", True).csv(
    "C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/features.csv"
)

joined_df = df.join(features_df, on=["Store", "Date", "IsHoliday"], how="left")

# Step 5: Extract year and perform analysis
with_year_df = joined_df.withColumn("Year", year(to_date(col("Date"), "yyyy-MM-dd")))
result_df = with_year_df.groupBy("Temperature", "Year") \
    .agg(sum("Weekly_Sales").alias("Total_Weekly_Sales")) \
    .orderBy("Temperature", "Year")

# Step 6: Write output to local file system
result_df.write.mode("overwrite").csv("C:/Users/ADIPAKSH/Desktop/Pyspark/output2-1/weekly_sales_by_temp_year.csv")

# Step 7: Write as Parquet and ORC
result_df.write.mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/Pyspark/output2-1/weekly_sales_parquet")
result_df.write.mode("overwrite").orc("C:/Users/ADIPAKSH/Desktop/Pyspark/output2-1/weekly_sales_orc")

# Step 8: Partition DF by Year
result_df.write.partitionBy("Year").mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/Pyspark/output2-1/partitioned_by_year")

# Step 9: Transformations and actions
filtered_df = result_df.filter(col("Total_Weekly_Sales") > 1000000)
filtered_df.show()
print(f"Count of filtered rows: {filtered_df.count()}")


# Optional: Stop Spark session
# spark.stop()



+-----------+----+------------------+
|Temperature|Year|Total_Weekly_Sales|
+-----------+----+------------------+
|       6.23|2011|1083071.1399999994|
|       9.55|2010|1034119.2099999997|
|      10.91|2011|1083657.6099999999|
|      11.17|2011|1565652.4700000002|
|      12.19|2011|1059715.2700000003|
|      14.48|2010|1001943.8000000002|
|       14.5|2011|        1159438.53|
|      14.56|2011|1179420.5000000005|
|      15.12|2011|1010711.0800000001|
|      15.25|2010|1364721.5799999998|
|      15.33|2012|        1146992.13|
|      15.47|2011|1070457.7999999996|
|      15.58|2011|        1110706.06|
|       16.6|2010|1149612.0399999998|
|      16.81|2011|1150003.3600000003|
|      16.94|2011|        2032864.77|
|       17.0|2011|         1206917.2|
|      17.05|2011|1548661.4500000004|
|      17.46|2011|1017056.4300000002|
|      17.91|2011|        1126921.34|
+-----------+----+------------------+
only showing top 20 rows

Count of filtered rows: 2852


In [27]:
# Code 3:- Weekly Sales By Stores Size(Gauri)

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col

# Step 1: Initialize SparkSession
spark = SparkSession.builder \
    .appName("WeeklySalesByStoreSize") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")

# Step 2: Load train.csv as RDD
rdd = spark.sparkContext.textFile("C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/train.csv")

# Step 3: Convert RDD to DataFrame
header = rdd.first()
data_rdd = rdd.filter(lambda row: row != header).map(lambda row: row.split(","))
df = data_rdd.map(lambda arr: (int(arr[0]), int(arr[1]), arr[2], float(arr[3]), arr[4] == 'TRUE')) \
    .toDF(["Store", "Dept", "Date", "Weekly_Sales", "IsHoliday"])

# Step 4: Load stores.csv to get Store Size
stores_df = spark.read.option("header", True).option("inferSchema", True).csv(
    "C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/stores.csv"
)

# Step 5: Join train data with store size
joined_df = df.join(stores_df.select("Store", "Size"), on="Store")

# Step 6: Group by Store Size and calculate total weekly sales
result_df = joined_df.groupBy("Size") \
    .agg(sum("Weekly_Sales").alias("Total_Weekly_Sales")) \
    .orderBy("Size")

# Step 7: Show result
result_df.show()

# Step 8: Write output to local file system
result_df.write.mode("overwrite").csv("C:/Users/ADIPAKSH/Desktop/Pyspark/output3-1/weekly_sales_by_store_size.csv")

# Step 9: Write as Parquet and ORC
result_df.write.mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/Pyspark/output3-1/weekly_sales_by_store_size_parquet")
result_df.write.mode("overwrite").orc("C:/Users/ADIPAKSH/Desktop/Pyspark/output3-1/weekly_sales_by_store_size_orc")

# Step 10: Partition by Size
result_df.write.partitionBy("Size").mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/Pyspark/output3-1/partitioned_by_size")



+------+--------------------+
|  Size|  Total_Weekly_Sales|
+------+--------------------+
| 34875|4.5475688900000155E7|
| 37392| 5.758673506999983E7|
| 39690|1.7188560080999953E8|
| 39910|1.7090804312999853E8|
| 41062| 9.056543541000025E7|
| 42988| 6.271688512000011E7|
| 57197| 7.425242539999999E7|
| 70713| 8.159827513999955E7|
| 93188|1.2778213882999982E8|
| 93638| 7.714155430999991E7|
|103681|1.3152067207999997E8|
|112238| 1.442872301499992E8|
|114533|1.9875061784999925E8|
|118221| 1.123953414199994E8|
|119557|1.4707564857000002E8|
|120653|1.5511473421000072E8|
|123737| 8.913368392000023E7|
|125833| 7.778921898999973E7|
|126512| 2.716177138899999E8|
|128107| 1.010611791700001E8|
+------+--------------------+
only showing top 20 rows



In [28]:
# Code 4:- Weekly Sales BY Store Type And Month(Gauri)
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, month, to_date

# Step 1: Initialize SparkSession
spark = SparkSession.builder \
    .appName("WeeklySalesByStoreTypeMonth") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")

# Step 2: Load train.csv as RDD
rdd = spark.sparkContext.textFile("C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/train.csv")

# Step 3: Convert RDD to DataFrame
header = rdd.first()
data_rdd = rdd.filter(lambda row: row != header).map(lambda row: row.split(","))
df = data_rdd.map(lambda arr: (int(arr[0]), int(arr[1]), arr[2], float(arr[3]), arr[4] == 'TRUE')) \
    .toDF(["Store", "Dept", "Date", "Weekly_Sales", "IsHoliday"])

# Step 4: Load stores.csv to get Store Type
stores_df = spark.read.option("header", True).option("inferSchema", True).csv(
    "C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/stores.csv"
)

# Step 5: Join train data with store type
joined_df = df.join(stores_df.select("Store", "Type"), on="Store")

# Step 6: Extract month from Date
with_month_df = joined_df.withColumn("Month", month(to_date(col("Date"), "yyyy-MM-dd")))

# Step 7: Group by Store Type and Month
result_df = with_month_df.groupBy("Type", "Month") \
    .agg(sum("Weekly_Sales").alias("Total_Weekly_Sales")) \
    .orderBy("Type", "Month")

# Step 8: Show result
result_df.show()

# Step 9: Write output to local file system
result_df.write.mode("overwrite").csv("C:/Users/ADIPAKSH/Desktop/Pyspark/output4-1/weekly_sales_by_store_type_month.csv")

# Step 10: Write as Parquet and ORC
result_df.write.mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/Pyspark/output4-1/weekly_sales_by_store_type_month_parquet")
result_df.write.mode("overwrite").orc("C:/Users/ADIPAKSH/Desktop/Pyspark/output4-1/weekly_sales_by_store_type_month_orc")

# Step 11: Partition by Type
result_df.write.partitionBy("Type").mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/Pyspark/output4-1/partitioned_by_type")


+----+-----+--------------------+
|Type|Month|  Total_Weekly_Sales|
+----+-----+--------------------+
|   A|    1|2.1417616809999934E8|
|   A|    2| 3.665076723800007E8|
|   A|    3|3.8077453309999824E8|
|   A|    4| 4.161801298799987E8|
|   A|    5| 3.590865958399991E8|
|   A|    6| 3.994480057699998E8|
|   A|    7| 4.172432103900018E8|
|   A|    8|3.9486368368999904E8|
|   A|    9| 3.731186221900001E8|
|   A|   10| 3.771314802799996E8|
|   A|   11|2.6472138660999936E8|
|   A|   12| 3.677632345200002E8|
|   B|    1| 9.544645479999976E7|
|   B|    2| 1.676717237399994E8|
|   B|    3|1.7513603761000013E8|
|   B|    4|1.9088059458999997E8|
|   B|    5|1.6345560924999985E8|
|   B|    6|1.8636250035999995E8|
|   B|    7|1.9374323200000003E8|
|   B|    8| 1.815049900400005E8|
+----+-----+--------------------+
only showing top 20 rows



In [36]:
#code 5:- Markdown Sales by year and store(Ayushi)
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, year, to_date
from pyspark.sql.types import DoubleType

# Step 1: Initialize SparkSession
spark = SparkSession.builder \
    .appName("MarkdownSalesByYearStore") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")

# Step 2: Load features.csv
features_df = spark.read.option("header", True).option("inferSchema", True).csv(
    "C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/features.csv"
)

# Step 3: Cast Markdown columns to DoubleType
for col_name in ["MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"]:
    features_df = features_df.withColumn(col_name, col(col_name).cast(DoubleType()))

# Step 4: Drop rows with nulls or NaNs in critical columns
cleaned_df = features_df.dropna(subset=[
    "Store", "Date", "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"
])

# Step 5: Extract year from Date
with_year_df = cleaned_df.withColumn("Year", year(to_date(col("Date"), "yyyy-MM-dd")))

# Step 6: Group and aggregate
markdown_df = with_year_df.groupBy("Store", "Year").agg(
    sum("MarkDown1").alias("Total_MarkDown1"),
    sum("MarkDown2").alias("Total_MarkDown2"),
    sum("MarkDown3").alias("Total_MarkDown3"),
    sum("MarkDown4").alias("Total_MarkDown4"),
    sum("MarkDown5").alias("Total_MarkDown5")
).orderBy("Store", "Year")

# Step 7: Show result
markdown_df.show()

# Step 8: Save outputs
markdown_df.write.mode("overwrite").csv("C:/Users/ADIPAKSH/Desktop/Pyspark/output5/markdown_sales_by_year_store.csv")
markdown_df.write.mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/Pyspark/output5/markdown_sales_by_year_store_parquet")
markdown_df.write.mode("overwrite").orc("C:/Users/ADIPAKSH/Desktop/Pyspark/output5/markdown_sales_by_year_store_orc")
markdown_df.write.partitionBy("Year").mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/Pyspark/output5/partitioned_by_year_markdown")

# Optional: Stop Spark session
# spark.stop()


+-----+----+------------------+------------------+------------------+------------------+------------------+
|Store|Year|   Total_MarkDown1|   Total_MarkDown2|   Total_MarkDown3|   Total_MarkDown4|   Total_MarkDown5|
+-----+----+------------------+------------------+------------------+------------------+------------------+
|    1|2011|40636.270000000004|          52673.92|          58818.12|           9800.41|          59518.63|
|    1|2012|345161.57999999996|         116615.72|          76716.44|         171364.42|171109.61000000002|
|    1|2013|261880.06999999998| 74836.09999999999|           6701.42|         112954.32| 93071.72000000002|
|    2|2011|53893.159999999996|         114448.38|          82233.17|16329.399999999998| 65380.59000000001|
|    2|2012| 463558.3599999999|173349.87000000005|107864.51999999999|208356.49000000002|233636.18000000008|
|    2|2013|         280722.32| 83392.78000000001| 7583.800000000001|132646.15999999997|142664.64000000004|
|    3|2011|          16530.

In [31]:
#Code 6:- Weekly Sales By Store Type(Ayushi)
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col

# Step 1: Initialize SparkSession
spark = SparkSession.builder \
    .appName("WeeklySalesByStoreType") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")

# Step 2: Load train.csv as RDD
rdd = spark.sparkContext.textFile("C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/train.csv")

# Step 3: Convert RDD to DataFrame
header = rdd.first()
data_rdd = rdd.filter(lambda row: row != header).map(lambda row: row.split(","))
df = data_rdd.map(lambda arr: (int(arr[0]), int(arr[1]), arr[2], float(arr[3]), arr[4] == 'TRUE')) \
    .toDF(["Store", "Dept", "Date", "Weekly_Sales", "IsHoliday"])

# Step 4: Load stores.csv to get Store Type
stores_df = spark.read.option("header", True).option("inferSchema", True).csv(
    "C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/stores.csv"
)

# Step 5: Join train data with store type
joined_df = df.join(stores_df.select("Store", "Type"), on="Store")

# Step 6: Group by Store Type and calculate total weekly sales
result_df = joined_df.groupBy("Type") \
    .agg(sum("Weekly_Sales").alias("Total_Weekly_Sales")) \
    .orderBy("Type")

# Step 7: Show result
result_df.show()

# Step 8: Write output to local file system
result_df.write.mode("overwrite").csv("C:/Users/ADIPAKSH/Desktop/Pyspark/output6/weekly_sales_by_store_type.csv")

# Step 9: Write as Parquet and ORC
result_df.write.mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/Pyspark/output6/weekly_sales_by_store_type_parquet")
result_df.write.mode("overwrite").orc("C:/Users/ADIPAKSH/Desktop/Pyspark/output6/weekly_sales_by_store_type_orc")

# Step 10: Partition by Type
result_df.write.partitionBy("Type").mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/Pyspark/output6/partitioned_by_type")


# Optional: Stop Spark session
# spark.stop()


+----+--------------------+
|Type|  Total_Weekly_Sales|
+----+--------------------+
|   A|4.3310147227500725E9|
|   B|2.0007007368200192E9|
|   C| 4.055035275399986E8|
+----+--------------------+



In [37]:
#Code 7:- Fuel Price By Year(Sanchit)
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, to_date, col, avg

# Step 1: Initialize SparkSession
spark = SparkSession.builder \
    .appName("FuelPriceByYear") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")

# Step 2: Load features.csv
features_df = spark.read.option("header", True).option("inferSchema", True).csv(
    "C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/features.csv"
)

# Step 3: Extract year from Date
with_year_df = features_df.withColumn("Year", year(to_date(col("Date"), "yyyy-MM-dd")))

# Step 4: Group by Year and calculate average fuel price
result_df = with_year_df.groupBy("Year") \
    .agg(avg("Fuel_Price").alias("Average_Fuel_Price")) \
    .orderBy("Year")

# Step 5: Show result
result_df.show()

# Step 6: Write output to local file system
result_df.write.mode("overwrite").csv("C:/Users/ADIPAKSH/Desktop/Pyspark/output7/fuel_price_by_year.csv")

# Step 7: Write as Parquet and ORC
result_df.write.mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/Pyspark/output7/fuel_price_by_year_parquet")
result_df.write.mode("overwrite").orc("C:/Users/ADIPAKSH/Desktop/Pyspark/output7/fuel_price_by_year_orc")

# Step 8: Partition by Year
result_df.write.partitionBy("Year").mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/Pyspark/output7/partitioned_by_year_fuel")

# Optional: Stop Spark session
# spark.stop()


+----+------------------+
|Year|Average_Fuel_Price|
+----+------------------+
|2010|2.8237671296296383|
|2011| 3.561914957264957|
|2012| 3.672084188034199|
|2013| 3.606057777777775|
+----+------------------+



In [39]:
#Code 8:- Weekly sales by year, month and date(Sanchit)
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, year, month, dayofmonth, col, sum

# Step 1: Initialize SparkSession
spark = SparkSession.builder \
    .appName("WeeklySalesByYearMonthDate") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")

# Step 2: Load train.csv as RDD
rdd = spark.sparkContext.textFile("C:/Users/ADIPAKSH/Downloads/walmart-recruiting-store-sales-forecasting_source TEAM4/train.csv")

# Step 3: Convert RDD to DataFrame
header = rdd.first()
data_rdd = rdd.filter(lambda row: row != header).map(lambda row: row.split(","))
df = data_rdd.map(lambda arr: (int(arr[0]), int(arr[1]), arr[2], float(arr[3]), arr[4] == 'TRUE')) \
    .toDF(["Store", "Dept", "Date", "Weekly_Sales", "IsHoliday"])

# Step 4: Extract Year, Month, and Day
with_date_df = df.withColumn("ParsedDate", to_date(col("Date"), "yyyy-MM-dd")) \
    .withColumn("Year", year(col("ParsedDate"))) \
    .withColumn("Month", month(col("ParsedDate"))) \
    .withColumn("Day", dayofmonth(col("ParsedDate")))

# Step 5: Group by Year, Month, and Date
result_df = with_date_df.groupBy("Year", "Month", "Date") \
    .agg(sum("Weekly_Sales").alias("Total_Weekly_Sales")) \
    .orderBy("Year", "Month", "Date")

# Step 6: Show result
result_df.show()

# Step 7: Write output to local file system
result_df.write.mode("overwrite").csv("C:/Users/ADIPAKSH/Desktop/Pyspark/output8/weekly_sales_by_year_month_date.csv")

# Step 8: Write as Parquet and ORC
result_df.write.mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/Pyspark/output8/weekly_sales_by_year_month_date_parquet")
result_df.write.mode("overwrite").orc("C:/Users/ADIPAKSH/Desktop/Pyspark/output8/weekly_sales_by_year_month_date_orc")

# Step 9: Partition by Year and Month
result_df.write.partitionBy("Year", "Month").mode("overwrite").parquet("C:/Users/ADIPAKSH/Desktop/Pyspark/output8/partitioned_by_year_month")


# Optional: Stop Spark session
# spark.stop()


+----+-----+----------+--------------------+
|Year|Month|      Date|  Total_Weekly_Sales|
+----+-----+----------+--------------------+
|2010|    2|2010-02-05|4.9750740500000045E7|
|2010|    2|2010-02-12| 4.833667763000007E7|
|2010|    2|2010-02-19| 4.827699377999997E7|
|2010|    2|2010-02-26|       4.396857113E7|
|2010|    3|2010-03-05| 4.687147029999999E7|
|2010|    3|2010-03-12| 4.592539651000002E7|
|2010|    3|2010-03-19|       4.498897464E7|
|2010|    3|2010-03-26| 4.413396105000002E7|
|2010|    4|2010-04-02|5.0423831259999976E7|
|2010|    4|2010-04-09|       4.736529044E7|
|2010|    4|2010-04-16|4.5183667079999976E7|
|2010|    4|2010-04-23| 4.473445256000005E7|
|2010|    4|2010-04-30| 4.370512670999997E7|
|2010|    5|2010-05-07|       4.850324352E7|
|2010|    5|2010-05-14| 4.533008019999998E7|
|2010|    5|2010-05-21| 4.512010805999997E7|
|2010|    5|2010-05-28| 4.775750255999994E7|
|2010|    6|2010-06-04| 5.018854312000005E7|
|2010|    6|2010-06-11| 4.782654671999997E7|
|2010|    